In [2]:
!git clone https://github.com/Mlawrence95/LinkedIn-Tech-Job-Data.git

Cloning into 'LinkedIn-Tech-Job-Data'...


In [1]:
!pip install -U pip setuptools wheel
!pip install spacy-transformers
#!pip install pytextrank
!python -m spacy download en_core_web_trf
!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_sm


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 2.1/2.1 MB 18.6 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 23.3 MB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
J:\Downloads\NLP\PyEnv\TorchEnv\Scripts\python.exe -m pip install -U pip setuptools wheel

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     -------------------------------------- 460.3/460.3 MB 7.4 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     --------------------------------------- 42.8/42.8 MB 10.6 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     --------------------------------------- 12.8/12.8 MB 14.9 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import csv
from utils import extract_wikidata

In [1]:
df = pd.read_csv('datasets/LinkedIn-Tech-Job-Data/jobs.csv')
records = df.to_dict(orient='records')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [32]:
jobs_info = [ 'post_id', 'industries','job_function', 'title', 'abstract', 'post_url', 'entity_info_title', 'entity_info_abstract']

#new_dict = []
djob = {}
wikidata_ids = []
with open('datasets/LinkedIn-Tech-Job-Data/jobs_new.csv', 'a', encoding='utf-8', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = jobs_info)
    #writer.writeheader()
    for job in records:
        djob["post_id"] = job['post_id']
        djob["industries"] = job['Industries']
        djob["job_function"] = job['Job function']
        djob["title"] = job['title']
        djob["abstract"] = extract_wikidata.extract_significant_sentences(job['description']) ##
        djob["post_url"] = job['post_url']

        #doc_trf = nlp_trf(job['description'])
        djob["entity_info_title"] = extract_wikidata.get_extract(job['description'], wikidata_ids)
        djob["entity_info_abstract"] = [] #remember to remove from util.py in for entity in eval(entity_info_abstract)
        #print(djob)
        writer.writerow(djob)


In [7]:
from utils.extract_wikidata import get_ids

wikidata_ids = []
with open('datasets/LinkedIn-Tech-Job-Data/jobs_new.csv', 'r', encoding='utf-8') as csv_file:
    df = pd.read_csv(csv_file)
    entity_info_title_col = df.entity_info_title

get_ids(entity_info_title_col, wikidata_ids)

In [11]:
len(wikidata_ids)

13398

In [10]:
file = open('datasets/LinkedIn-Tech-Job-Data/wikidata_ids.txt', 'a', encoding='utf-8')
for wikidata in wikidata_ids:
	file.write(wikidata + "\n")
file.close()

#news | jobs
#id : post_id
#category: Industries
#subcategory : job function
#title: title
#abstract : requirements
#url : url
#entity_info_title : only one [{label, type, wikidataId, confidence, occurrence offsets, surface forms}]
#entity_info_abstract: all the first 3[{label, type, wikidataId, confidence, occurrence offsets, surface forms}]


data:
  train_news : "./datasets/mind_train_demo/news.tsv" ###
  train_behavior : "./datasets/mind_train_demo/behaviors.tsv"
  valid_news : "./datasets/mind_val_demo/news.tsv"
  valid_behavior : "./datasets/mind_val_demo/behaviors.tsv"
  knowledge_graph : "./datasets/mind_utils_demo/triple2id.txt" --- h t r
  entity_embedding : "./datasets/mind_utils_demo/entity2vecd100.vec" ###
  num_entity_embedding: 3275150
  relation_embedding : "./datasets/mind_utils_demo/relation2vecd100.vec" ---
  entity_index : "./datasets/mind_utils_demo/entity2id.txt" ###
  relation_index : "./datasets/mind_utils_demo/relation2id.txt" ---
  sparse_adj_entity : "./datasets/mind_utils_demo/entity_adj.npy"
  sparse_adj_relation : "./datasets/mind_utils_demo/relation_adj.npy"
  mind_data : "./data_dict_1672848868384.pkl"

In [3]:
import numpy as np
import pandas as pd
with open('datasets/LinkedIn-Tech-Job-Data/jobs_new.csv', 'r', encoding='utf-8') as csv_file:
    # Load the data from the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)

    # Calculate the number of rows in the DataFrame
    n = df.shape[0]

    # Generate a random permutation of the indices
    idx = np.random.permutation(n)

    # Split the indices into training and validation sets
    split = int(0.8 * n)
    train_idx = idx[:split]
    val_idx = idx[split:]

    # Split the DataFrame into training and validation sets based on the indices
    train_df = df.iloc[train_idx, :]
    val_df = df.iloc[val_idx, :]

    # Save the training and validation sets as CSV files
    train_df.to_csv('datasets/LinkedIn-Tech-Job-Data/train_jobs.tsv',  sep='\t', index=False)
    val_df.to_csv('datasets/LinkedIn-Tech-Job-Data/valid_jobs.tsv',  sep='\t', index=False)

In [4]:
# Config Generation
import time
import torch
import os
from utils import *
from utils.cleaner_jobs import *
from train_test import *
import argparse
from parse_config import ConfigParser

parser = argparse.ArgumentParser(description='KRED')
parser.add_argument('-f')
parser.add_argument('-c', '--config', default="./config.json", type=str,
                    help='config file path (default: None)')
parser.add_argument('-r', '--resume', default=None, type=str,
                    help='path to latest checkpoint (default: None)')
parser.add_argument('-d', '--device', default=None, type=str,
                    help='indices of GPUs to enable (default: all)')

config = ConfigParser.from_args(parser)

In [2]:

config = cleaner_jobs(config)

0
{}
1
[{"Label": "Artificial Intelligence", "WikidataId": "Q11660"}, {"Label": "Public Services", "WikidataId": "Q15817798"}, {"Label": "Data Scientist", "WikidataId": "Q29169143"}, {"Label": "ETL", "WikidataId": "Q1276130", "OccurrenceOffsets": 946}, {"Label": "Deloitte", "WikidataId": "Q491748", "OccurrenceOffsets": 1276}, {"Label": "GPS", "WikidataId": "Q18822", "OccurrenceOffsets": 1319}, {"Label": "Federal", "WikidataId": "Q183", "OccurrenceOffsets": 2433}, {"Label": "Python", "WikidataId": "Q28865", "OccurrenceOffsets": 2726}, {"Label": "Java", "WikidataId": "Q251", "OccurrenceOffsets": 2734}, {"Label": "Tableau", "WikidataId": "Q546808", "OccurrenceOffsets": 2804}, {"Label": "Qlik", "WikidataId": "Q923062", "OccurrenceOffsets": 2813}, {"Label": "PowerBI", "WikidataId": "Q23542287", "OccurrenceOffsets": 2819}, {"Label": "SQL", "WikidataId": "Q47607", "OccurrenceOffsets": 2889}, {"Label": "United States", "WikidataId": "Q30"}, {"Label": "Master", "WikidataId": "Q183816", "Occurre

JSONDecodeError: Expecting ',' delimiter: line 1 column 73 (char 72)

In [26]:
config["jobs"]["train_jobs"]

'./datasets/LinkedIn-Tech-Job-Data/train_jobs.tsv'

In [6]:
escape_quote(r{"[{'Label': "O\'\Reilly Auto Parts", \'WikidataId\': ""Q7071951""}]"})

'[{"Label": O"Reilly Auto Parts, "WikidataId": Q7071951}]'